In [6]:
from PyPDF2 import PdfWriter, PdfReader
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.lib import colors
import io

def add_page_numbers(input_pdf, output_pdf, page_number_color=colors.black):
    input_pdf_reader = PdfReader(open(input_pdf, "rb"))
    output_pdf_writer = PdfWriter()
    total_pages = len(input_pdf_reader.pages)

    for page_num in range(total_pages):
        # get the dimensions of the current page
        # and calculate font size based on page dimensions
        page = input_pdf_reader.pages[page_num]
        page_width = page.mediabox.width
        page_height = page.mediabox.height
        font_size = float(min(page_width, page_height)) * 0.02  # 2% of the smaller dimension

        # create a PDF with the page number
        packet = io.BytesIO()
        can = canvas.Canvas(packet, pagesize=letter)
        can.setFillColor(page_number_color)  # set the color of the page number
        can.setFont("Courier", font_size)  # Set the font size
        can.drawString(10, 10, f'{page_num + 1}/{total_pages}')  # position the page number
        can.save()

        # move to the beginning of the StringIO buffer
        packet.seek(0)
        page_number_pdf = PdfReader(packet)

        # merge the page number pdf with the input pdf page
        page = input_pdf_reader.pages[page_num]
        page.merge_page(page_number_pdf.pages[0])
        output_pdf_writer.add_page(page)

    with open(output_pdf, "wb") as output_file:
        output_pdf_writer.write(output_file)

input_pdf = "input/combined.pdf"
output_pdf = "output/numbered.pdf"
add_page_numbers(input_pdf, output_pdf, page_number_color=colors.blueviolet)